In [10]:
!pip install pymysql
!pip install tensorflow

In [40]:
import tensorflow as tf
import pandas as pd
import pymysql

#모델 불러오기
model = tf.keras.models.load_model('fire_anomaly_model.h5')

#데이터 베이스 연결
def dbconnect() : 
    conn = pymysql.connect(host = 'localhost', port = 3306, user = 'test1', password = 'P@ssw0rd', db = 'test', charset='utf8')
    return conn

# 데이터 로드 전처리
def fetch_data(conn) :
    # 데이터 로드
    query = "SELECT * FROM sensor_data"
    df = pd.read_sql(query, conn)
    # 날짜 순으로 소트
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.sort_values('Date').reset_index(drop=True)
    # 사용 데이터 지정
    features = ['CO_Room', 'Temperature_Room', 'Humidity_Room', 'PM10_Room']
    # 결측치 제거
    df = df.dropna(subset=features)
    # CO 음수값 제거
    df['CO_Room'] = df['CO_Room'].clip(lower=0)    
    return df

# 시퀸스 생성 (시계열 윈도우 입력 요구)
def create_sequences(df, features, window_size=60):
    from sklearn.preprocessing import RobustScaler
    import numpy as np

    # 정규화
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(df[features])
    
    # 시퀀스 생성
    X_seq = []
    for i in range(len(X_scaled) - window_size):
        X_seq.append(X_scaled[i:i + window_size])
    
    return np.array(X_seq)

In [44]:
conn = dbconnect()
df = fetch_data(conn)
conn.close()

X_seq = create_sequences(df, features=['CO_Room', 'Temperature_Room', 'Humidity_Room', 'PM10_Room'])

# 최신 시퀀스 1개 예측
latest_X = X_seq[-1].reshape(1, 60, 4)
prob = model.predict(latest_X)[0][0]

print(f"🔥 화재 확률: {prob:.4f} → {'경고' if prob > 0.5 else '정상'}")

C:\Users\USER1\AppData\Local\Temp\ipykernel_10988\2126530840.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
🔥 화재 확률: 0.9989 → 경고


In [50]:
import time

while True:
    conn = dbconnect()
    df = fetch_data(conn)
    conn.close()

    X_seq = create_sequences(df, features=['CO_Room', 'Temperature_Room', 'Humidity_Room', 'PM10_Room'])
    latest_X = X_seq[-1].reshape(1, 60, 4)
    prob = model.predict(latest_X)[0][0]

    print(f"[{pd.Timestamp.now()}] 화재 예측 확률: {prob:.4f}")
    
    time.sleep(30)  # 30초마다 실행

C:\Users\USER1\AppData\Local\Temp\ipykernel_10988\2126530840.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[2025-06-18 14:14:11.670431] 화재 예측 확률: 0.9989


C:\Users\USER1\AppData\Local\Temp\ipykernel_10988\2126530840.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[2025-06-18 14:14:50.286507] 화재 예측 확률: 0.9989


KeyboardInterrupt: 

## 위 코드가 정상 코드. 아래는 시뮬레이션 코드들

In [59]:
### 있는 데이터를 30초 간격으로 불러오는 시뮬레이션

import tensorflow as tf
import pandas as pd
import pymysql
from sklearn.preprocessing import RobustScaler
import numpy as np
import time

# 모델 불러오기
model = tf.keras.models.load_model('fire_anomaly_model.h5')

# DB 연결
def dbconnect():
    conn = pymysql.connect(
        host='localhost',
        port=3306,
        user='test1',
        password='P@ssw0rd',
        db='test',
        charset='utf8'
    )
    return conn

# 데이터 로드 및 전처리
def fetch_data(conn):
    query = "SELECT * FROM sensor_data"
    df = pd.read_sql(query, conn)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.sort_values('Date').reset_index(drop=True)
    
    features = ['CO_Room', 'Temperature_Room', 'Humidity_Room', 'PM10_Room']
    df = df.dropna(subset=features)
    df['CO_Room'] = df['CO_Room'].clip(lower=0)

    return df, features

# 시퀀스 생성 (전체 시퀀스를 미리 만듦)
def create_sequences(df, features, window_size=60):
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(df[features])
    X_seq = []
    for i in range(len(X_scaled) - window_size):
        X_seq.append(X_scaled[i:i + window_size])
    return np.array(X_seq)

# 1. DB에서 전체 데이터 불러오기
conn = dbconnect()
df, features = fetch_data(conn)
conn.close()

# 2. 시퀀스 생성
X_seq = create_sequences(df, features)

# 3. 순차적 예측 시뮬레이션
print(f"총 시퀀스 개수: {len(X_seq)}\n시뮬레이션 시작...")

for i in range(len(X_seq)):
    input_seq = X_seq[i].reshape(1, 60, 4)  # (1, 60, 4)
    prob = model.predict(input_seq)[0][0]
    label = int(prob > 0.5)
    print(f"[{i+1}/{len(X_seq)}] 🔥 화재 확률: {prob:.4f} → {'🚨 경고' if label == 1 else '✅ 정상'}")
    
    time.sleep(30)  # 30초 간격으로 순차 예측

C:\Users\USER1\AppData\Local\Temp\ipykernel_10988\3929465159.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


총 시퀀스 개수: 305244
시뮬레이션 시작...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
[1/305244] 🔥 화재 확률: 0.3077 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[2/305244] 🔥 화재 확률: 0.3044 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[3/305244] 🔥 화재 확률: 0.3037 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[4/305244] 🔥 화재 확률: 0.3667 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[5/305244] 🔥 화재 확률: 0.4243 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
[6/305244] 🔥 화재 확률: 0.4460 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
[7/305244] 🔥 화재 확률: 0.5160 → 🚨 경고
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[8/305244] 🔥 화재 확률: 0.5484 → 🚨 경고
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[9/305244] 🔥 화재 확률: 0.5628 → 🚨 경고
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[10/305244] 🔥 화재 확률: 0.5697 → 🚨 경고
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
[11/305244] 🔥 화재 확률: 0.5747 → 🚨 경고
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[12/305244] 🔥 화재 확률: 0.5752 → 🚨 경고


KeyboardInterrupt: 

In [ ]:
## 데이터 불러오는 간격 1초 좁힘 + X_seq 80% 이상 일때 라벨 1 

import tensorflow as tf
import pandas as pd
import pymysql
from sklearn.preprocessing import RobustScaler
import numpy as np
import time

# 모델 불러오기
model = tf.keras.models.load_model('fire_anomaly_model.h5')

# DB 연결
def dbconnect():
    conn = pymysql.connect(
        host='localhost',
        port=3306,
        user='test1',
        password='P@ssw0rd',
        db='test',
        charset='utf8'
    )
    return conn

# 데이터 로드 및 전처리
def fetch_data(conn):
    query = "SELECT * FROM sensor_data"
    df = pd.read_sql(query, conn)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.sort_values('Date').reset_index(drop=True)
    
    features = ['CO_Room', 'Temperature_Room', 'Humidity_Room', 'PM10_Room']
    df = df.dropna(subset=features)
    df['CO_Room'] = df['CO_Room'].clip(lower=0)

    return df, features

# 시퀀스 생성 (전체 시퀀스를 미리 만듦)
def create_sequences(df, features, window_size=60):
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(df[features])
    X_seq = []
    for i in range(len(X_scaled) - window_size):
        X_seq.append(X_scaled[i:i + window_size])
    return np.array(X_seq)

# DB에서 전체 데이터 불러오기
conn = dbconnect()
df, features = fetch_data(conn)
conn.close()

# 시퀀스 생성
X_seq = create_sequences(df, features)

# 순차적 예측 시뮬레이션
print(f"총 시퀀스 개수: {len(X_seq)}\n시뮬레이션 시작...")

for i in range(len(X_seq)):
    input_seq = X_seq[i].reshape(1, 60, 4)  # (1, 60, 4)
    prob = model.predict(input_seq)[0][0]
    label = int(prob > 0.8)
    print(f"[{i+1}/{len(X_seq)}] 🔥 화재 확률: {prob:.4f} → {'🚨 경고' if label == 1 else '✅ 정상'}")

    if label == 1:
        print("\n🚨 경고 발생 시점의 원본 데이터 (최근 60개):")
        window_df = df.iloc[i:i+60][['Date'] + features]
        print(window_df.tail(60))  # 60개
    
    time.sleep(1)  # 1초 간격으로 순차 예측

C:\Users\USER1\AppData\Local\Temp\ipykernel_10988\4065921714.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


총 시퀀스 개수: 305244
시뮬레이션 시작...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
[1/305244] 🔥 화재 확률: 0.3077 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[2/305244] 🔥 화재 확률: 0.3044 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
[3/305244] 🔥 화재 확률: 0.3037 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[4/305244] 🔥 화재 확률: 0.3667 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[5/305244] 🔥 화재 확률: 0.4243 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
[6/305244] 🔥 화재 확률: 0.4460 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[7/305244] 🔥 화재 확률: 0.5160 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[8/305244] 🔥 화재 확률: 0.5484 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[9/305244] 🔥 화재 확률: 0.5628 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[10/305244] 🔥 화재 확률: 0.5697 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[11/305244] 🔥 화재 확률: 0.5747 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
[12/305244] 🔥 화재 확률: 0.5752 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
[13/305244] 🔥 화재 확률: 0.5407 → ✅ 정상
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48